In [4]:
readlines("input")

100-element Vector{String}:
 "4762999789865789979838469132535949928924821629179961723363249471951799931826691953949998875699647749"
 "2878689389148477519999123173439839916421859766789398177792745399987989799399369898869962879881857886"
 "4681844686577988929825996716999999595142263859539675899257971592875674325292821698696879525129998942"
 "9336787159699929348236396957149575286918149176919845175396915979658979864871977981679794958877594678"
 "4412778865289876989817697439991158316498888846389869221481946188359399391987999799997669499861717661"
 "1439891758972697742611999788912917891385861194286537969159929729819269822367391926215581467965586992"
 "3352872759759591611957718288411915637128687168996939959672784873924132978747687969491913828248378578"
 "6291982518192342881785738844996188999668491931829567896518882959594942969816195948899281728139998788"
 "9296771938999197744892679699868148825285397967775946749986154979839885323669598796289775678988988699"
 "8844945987918794881952985499178597

In [6]:
# field is transposed but I don't care 🙂
field = parse.(Int, reduce(hcat,collect.(readlines("input"))))

100×100 Matrix{Int64}:
 4  2  4  9  4  1  3  6  9  8  9  9  9  …  1  8  8  9  9  5  9  8  8  7  9  4
 7  8  6  3  4  4  3  2  2  8  8  3  9     6  9  9  6  2  6  9  8  9  7  5  8
 6  7  8  3  1  3  5  9  9  4  4  9  5     9  8  1  9  8  4  5  9  2  5  9  6
 2  8  1  6  2  9  2  1  6  4  5  9  9     8  2  8  8  9  9  5  6  9  8  6  8
 9  6  8  7  7  8  8  9  7  9  9  9  9     5  2  9  7  9  7  9  9  1  4  8  7
 9  8  4  8  7  9  7  8  7  4  5  9  2  …  3  5  9  5  8  9  9  7  9  9  9  4
 9  9  4  7  8  1  2  2  1  5  9  9  6     7  6  4  8  8  2  6  2  7  3  9  3
 7  3  6  1  8  7  7  5  9  9  2  8  9     9  4  6  1  3  1  2  4  6  9  1  3
 8  8  8  5  6  5  5  1  3  8  9  7  8     8  7  6  9  8  5  2  1  9  8  8  9
 9  9  6  9  5  8  9  8  8  7  5  1  2     4  1  2  9  9  9  3  7  5  5  9  4
 8  1  5  6  2  9  7  1  9  9  5  9  9  …  3  4  8  8  7  7  8  7  6  8  2  8
 6  4  7  9  8  7  5  9  9  1  5  2  5     9  1  9  3  4  1  9  9  8  2  4  2
 5  8  7  9  9  2  9  2  9  8  8  9  5   

Again it looks like dynamic programming but there is no restriction to move either right or left so again it won't work. Instead I use a shortest path algorithm. Could use a graph package for that but for practice I'll implement Dijkstra myself.

In [11]:
a = [1,3,5,2,0,6]
sort!(a, rev=true)
pop!(a)

0

In [22]:
# init
to_explore = [(x,y) for x in 1:100 for y in 1:100]
scores = Dict()
for pos in to_explore
    scores[pos] = Inf
end
scores[(1,1)] = 0

0

In [23]:
while length(to_explore) > 0
    sort!(to_explore, by = x->scores[x], rev=true)
    pos = pop!(to_explore)
    score = scores[pos]
    for (i,j) in zip([1,-1,0,0], [0,0,1,-1])
        if pos[1]+i>0 && pos[1]+i<101 && pos[2]+j>0 && pos[2]+j<101
            new_pos = (pos[1]+i, pos[2]+j)
            s = score + field[new_pos[1],new_pos[2]]
            if !haskey(scores, new_pos) || s < scores[new_pos]
                scores[new_pos] = s
            end
        end
    end
    if length(to_explore)%1000 == 0
        println(length(to_explore))
    end
end                

9000
8000
7000
6000
5000
4000
3000
2000
1000
0


In [24]:
scores[100,100]

745

## Part II

Same problem 25x larger. I'm afraid it might be very slow. Still let's try the brute-force variant first.

In [25]:
field

100×100 Matrix{Int64}:
 4  2  4  9  4  1  3  6  9  8  9  9  9  …  1  8  8  9  9  5  9  8  8  7  9  4
 7  8  6  3  4  4  3  2  2  8  8  3  9     6  9  9  6  2  6  9  8  9  7  5  8
 6  7  8  3  1  3  5  9  9  4  4  9  5     9  8  1  9  8  4  5  9  2  5  9  6
 2  8  1  6  2  9  2  1  6  4  5  9  9     8  2  8  8  9  9  5  6  9  8  6  8
 9  6  8  7  7  8  8  9  7  9  9  9  9     5  2  9  7  9  7  9  9  1  4  8  7
 9  8  4  8  7  9  7  8  7  4  5  9  2  …  3  5  9  5  8  9  9  7  9  9  9  4
 9  9  4  7  8  1  2  2  1  5  9  9  6     7  6  4  8  8  2  6  2  7  3  9  3
 7  3  6  1  8  7  7  5  9  9  2  8  9     9  4  6  1  3  1  2  4  6  9  1  3
 8  8  8  5  6  5  5  1  3  8  9  7  8     8  7  6  9  8  5  2  1  9  8  8  9
 9  9  6  9  5  8  9  8  8  7  5  1  2     4  1  2  9  9  9  3  7  5  5  9  4
 8  1  5  6  2  9  7  1  9  9  5  9  9  …  3  4  8  8  7  7  8  7  6  8  2  8
 6  4  7  9  8  7  5  9  9  1  5  2  5     9  1  9  3  4  1  9  9  8  2  4  2
 5  8  7  9  9  2  9  2  9  8  8  9  5   

In [28]:
using Lazy

In [29]:
next_tile(field) = field .% 9 .+ 1

next_tile (generic function with 1 method)

In [32]:
tiles = iterated(next_tile,field)

List: ([4 2 … 9 4; 7 8 … 5 8; … ; 4 8 … 9 3; 9 6 … 5 9] [5 3 … 1 5; 8 9 … 6 9; … ; 5 9 … 1 4; 1 7 … 6 1] [6 4 … 2 6; 9 1 … 7 1; … ; 6 1 … 2 5; 2 8 … 7 2] [7 5 … 3 7; 1 2 … 8 2; … ; 7 2 … 3 6; 3 9 … 8 3] [8 6 … 4 8; 2 3 … 9 3; … ; 8 3 … 4 7; 4 1 … 9 4] [9 7 … 5 9; 3 4 … 1 4; … ; 9 4 … 5 8; 5 2 … 1 5] [1 8 … 6 1; 4 5 … 2 5; … ; 1 5 … 6 9; 6 3 … 2 6] [2 9 … 7 2; 5 6 … 3 6; … ; 2 6 … 7 1; 7 4 … 3 7] [3 1 … 8 3; 6 7 … 4 7; … ; 3 7 … 8 2; 8 5 … 4 8] [4 2 … 9 4; 7 8 … 5 8; … ; 4 8 … 9 3; 9 6 … 5 9] [5 3 … 1 5; 8 9 … 6 9; … ; 5 9 … 1 4; 1 7 … 6 1] …)

In [38]:
tiles[2]

100×100 Matrix{Int64}:
 5  3  5  1  5  2  4  7  1  9  1  1  1  …  2  9  9  1  1  6  1  9  9  8  1  5
 8  9  7  4  5  5  4  3  3  9  9  4  1     7  1  1  7  3  7  1  9  1  8  6  9
 7  8  9  4  2  4  6  1  1  5  5  1  6     1  9  2  1  9  5  6  1  3  6  1  7
 3  9  2  7  3  1  3  2  7  5  6  1  1     9  3  9  9  1  1  6  7  1  9  7  9
 1  7  9  8  8  9  9  1  8  1  1  1  1     6  3  1  8  1  8  1  1  2  5  9  8
 1  9  5  9  8  1  8  9  8  5  6  1  3  …  4  6  1  6  9  1  1  8  1  1  1  5
 1  1  5  8  9  2  3  3  2  6  1  1  7     8  7  5  9  9  3  7  3  8  4  1  4
 8  4  7  2  9  8  8  6  1  1  3  9  1     1  5  7  2  4  2  3  5  7  1  2  4
 9  9  9  6  7  6  6  2  4  9  1  8  9     9  8  7  1  9  6  3  2  1  9  9  1
 1  1  7  1  6  9  1  9  9  8  6  2  3     5  2  3  1  1  1  4  8  6  6  1  5
 9  2  6  7  3  1  8  2  1  1  6  1  1  …  4  5  9  9  8  8  9  8  7  9  3  9
 7  5  8  1  9  8  6  1  1  2  6  3  6     1  2  1  4  5  2  1  1  9  3  5  3
 6  9  8  1  1  3  1  3  1  9  9  1  6   

In [43]:
full_field = [
 tiles[1] tiles[2] tiles[3] tiles[4] tiles[5];
 tiles[2] tiles[3] tiles[4] tiles[5] tiles[6];
 tiles[3] tiles[4] tiles[5] tiles[6] tiles[7];
 tiles[4] tiles[5] tiles[6] tiles[7] tiles[8];
 tiles[5] tiles[6] tiles[7] tiles[8] tiles[9];
]

500×500 Matrix{Int64}:
 4  2  4  9  4  1  3  6  9  8  9  9  9  …  5  3  3  4  4  9  4  3  3  2  4  8
 7  8  6  3  4  4  3  2  2  8  8  3  9     1  4  4  1  6  1  4  3  4  2  9  3
 6  7  8  3  1  3  5  9  9  4  4  9  5     4  3  5  4  3  8  9  4  6  9  4  1
 2  8  1  6  2  9  2  1  6  4  5  9  9     3  6  3  3  4  4  9  1  4  3  1  3
 9  6  8  7  7  8  8  9  7  9  9  9  9     9  6  4  2  4  2  4  4  5  8  3  2
 9  8  4  8  7  9  7  8  7  4  5  9  2  …  7  9  4  9  3  4  4  2  4  4  4  8
 9  9  4  7  8  1  2  2  1  5  9  9  6     2  1  8  3  3  6  1  6  2  7  4  7
 7  3  6  1  8  7  7  5  9  9  2  8  9     4  8  1  5  7  5  6  8  1  4  5  7
 8  8  8  5  6  5  5  1  3  8  9  7  8     3  2  1  4  3  9  6  5  4  3  3  4
 9  9  6  9  5  8  9  8  8  7  5  1  2     8  5  6  4  4  4  7  2  9  9  4  8
 8  1  5  6  2  9  7  1  9  9  5  9  9  …  7  8  3  3  2  2  3  2  1  3  6  3
 6  4  7  9  8  7  5  9  9  1  5  2  5     4  5  4  7  8  5  4  4  3  6  8  6
 5  8  7  9  9  2  9  2  9  8  8  9  5   

I can re-use scores from part I. Also it makes more sense to dynamically expand the list of `to_explore` instead of adding all positions with a score of `Inf`. This avoids sorting a large list in every iteration.

In [50]:
vcat([1,2,3],[3,2])

5-element Vector{Int64}:
 1
 2
 3
 3
 2

In [56]:
to_explore = unique(vcat([(100,x) for x in 1:100], [(x,100) for x in 1:100]))

199-element Vector{Tuple{Int64, Int64}}:
 (100, 1)
 (100, 2)
 (100, 3)
 (100, 4)
 (100, 5)
 (100, 6)
 (100, 7)
 (100, 8)
 (100, 9)
 (100, 10)
 (100, 11)
 (100, 12)
 (100, 13)
 ⋮
 (88, 100)
 (89, 100)
 (90, 100)
 (91, 100)
 (92, 100)
 (93, 100)
 (94, 100)
 (95, 100)
 (96, 100)
 (97, 100)
 (98, 100)
 (99, 100)

In [58]:
while length(to_explore) > 0
    sort!(to_explore, by = x->scores[x], rev=true)
    pos = pop!(to_explore)
    score = scores[pos]
    for (i,j) in zip([1,-1,0,0], [0,0,1,-1])
        if pos[1]+i>0 && pos[1]+i<501 && pos[2]+j>0 && pos[2]+j<501
            new_pos = (pos[1]+i, pos[2]+j)
            s = score + full_field[new_pos[1],new_pos[2]]
            if !haskey(scores, new_pos)
                scores[new_pos] = s
                push!(to_explore, new_pos)
            elseif s < scores[new_pos]
                scores[new_pos] = s
            end
        end
    end
    if length(scores)%1000 == 0
        println(length(scores),"\t",length(to_explore))
    end
end 

13000	275
14000	323
17000	369
17000	368
19000	381
20000	401
21000	459
24000	455
25000	506
28000	446
29000	490
31000	512
32000	493
33000	498
37000	544
38000	549
38000	548
38000	547
41000	546
43000	562
44000	566
44000	565
45000	620
46000	609
46000	608
47000	591
47000	590
49000	605
50000	593
51000	597
52000	622
52000	621
54000	612
55000	627
56000	629
56000	628
56000	627
57000	624
59000	697
59000	696
59000	695
59000	694
59000	693
59000	692
60000	682
60000	681
64000	674
66000	654
67000	681
69000	707
70000	688
71000	705
72000	688
74000	714
74000	713
76000	659
78000	720
80000	716
82000	731
85000	724
85000	723
86000	744
87000	730
89000	741
90000	746
91000	763
92000	733
95000	751
95000	750
95000	749
95000	748
95000	747
95000	746
96000	768
97000	764
101000	739
102000	783
103000	806
106000	814
107000	763
108000	761
109000	734
110000	746
110000	745
110000	744
110000	743
110000	742
111000	785
111000	784
114000	785
115000	824
116000	809
116000	808
117000	834
118000	796
120000	821
121000	784
127000	7

In [59]:
scores[(500,500)]

3002